Looking at the Hopkins COVID-19 data that was cloned from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import statistics
from datetime import datetime as dt

df_confirmed = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

#df_deaths = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
#df_recovered = pd.read_csv('./csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [2]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,990,990,990,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,410,411,413,414,414,418,418,422,426,428
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,296,296,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,102,119,120,124


In [4]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 51 columns):
Province/State    155 non-null object
Country/Region    255 non-null object
Lat               255 non-null float64
Long              255 non-null float64
1/22/20           255 non-null int64
1/23/20           255 non-null int64
1/24/20           255 non-null int64
1/25/20           255 non-null int64
1/26/20           255 non-null int64
1/27/20           255 non-null int64
1/28/20           255 non-null int64
1/29/20           255 non-null int64
1/30/20           255 non-null int64
1/31/20           255 non-null int64
2/1/20            255 non-null int64
2/2/20            255 non-null int64
2/3/20            255 non-null int64
2/4/20            255 non-null int64
2/5/20            255 non-null int64
2/6/20            255 non-null int64
2/7/20            255 non-null int64
2/8/20            255 non-null int64
2/9/20            255 non-null int64
2/10/20           255 non-null int64
2

I think I just need to work with the confirmed DataFrame. It's set up such that each row is a specific location, and the columns are different dates.



In [5]:
#creating template for dataframes to record various values

df_template = df_confirmed.copy()


df_template.head(), df_template.size

(  Province/State  Country/Region      Lat      Long  1/22/20  1/23/20  \
 0          Anhui  Mainland China  31.8257  117.2264        1        9   
 1        Beijing  Mainland China  40.1824  116.4142       14       22   
 2      Chongqing  Mainland China  30.0572  107.8740        6        9   
 3         Fujian  Mainland China  26.0789  117.9874        1        5   
 4          Gansu  Mainland China  36.0611  103.8343        0        2   
 
    1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/20  2/29/20  3/1/20  3/2/20  \
 0       15       39       60       70  ...      990      990     990     990   
 1       36       41       68       80  ...      410      411     413     414   
 2       27       57       75      110  ...      576      576     576     576   
 3       10       18       35       59  ...      296      296     296     296   
 4        2        4        7       14  ...       91       91      91      91   
 
    3/3/20  3/4/20  3/5/20  3/6/20  3/7/20  3/8/20  
 0     990   

In [11]:
# Creating template out of JHU table format

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

for x in df_template.columns.values:
    if (x not in do_not_include):
        for y in range(df_template['1/22/20'].values.size):
            df_template.at[y, x] = 0
            
            
df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,1/1/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Adding the rest of the beginning of January

for q in range(1, 22):
    day_str = str(q)
    date_str = '1/' + day_str + '/20'
    df_template[date_str] = 0    

df_template.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 72 columns):
Province/State    155 non-null object
Country/Region    255 non-null object
Lat               255 non-null float64
Long              255 non-null float64
1/22/20           255 non-null int64
1/23/20           255 non-null int64
1/24/20           255 non-null int64
1/25/20           255 non-null int64
1/26/20           255 non-null int64
1/27/20           255 non-null int64
1/28/20           255 non-null int64
1/29/20           255 non-null int64
1/30/20           255 non-null int64
1/31/20           255 non-null int64
2/1/20            255 non-null int64
2/2/20            255 non-null int64
2/3/20            255 non-null int64
2/4/20            255 non-null int64
2/5/20            255 non-null int64
2/6/20            255 non-null int64
2/7/20            255 non-null int64
2/8/20            255 non-null int64
2/9/20            255 non-null int64
2/10/20           255 non-null int64
2

In [9]:
# For description of the different units being captured from the API,
# please see https://darksky.net/dev/docs
# Requests are being made in SI units

# dataframe to record median maximum temperature of preceding 14 days
df_tMax = df_template.copy()

# dataframe to record median minimum temperature of preceding 14 days
df_tMin = df_template.copy()

# dataframe to record median humidity of preceding 14 days
df_humidity = df_template.copy()

# dataframe to record median UV index of preceding 14 days
df_uvIndex = df_template.copy()

# dataframe to record median cloud cover of preceding 14 days
df_cloud = df_template.copy()

# dataframe to record weather summaries of the preceding 14 days
#df_summaries = df_template.copy()

# dataframe to record precipitation probability of the preceding 14 days
df_precipprob = df_template.copy()

# dataframe to record dewpoint of the preceding 14 days
df_dewpoint = df_template.copy()

# dataframe to record pressure of the preceding 14 days
df_pressure = df_template.copy()

# dataframe to record windspeed of the preceding 14 days
df_windspeed = df_template.copy()

# dataframe to record "ozone" reading of the preceding 14 days
df_ozone = df_template.copy()


In [ ]:
#Darksky API key
API_KEY = 'API KEY'

#number of seconds in one day
one_day = 86400

t = '1579651200'

latitude = '31.8257'
longitude = '117.2264'

# sample get request 
#GET https://api.darksky.net/forecast/0123456789abcdef9876543210fedcba/
#42.3601,-71.0589,255657600?exclude=currently,flags

url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + t

url = url + '?exclude=currently,flags&units=si'
url

In [15]:
response = requests.get(url)

In [ ]:
response.content

In [51]:
info = json.loads(response.content)

info['daily']['data'][0]

{'time': 1579622400,
 'summary': 'Light rain in the morning and afternoon.',
 'icon': 'rain',
 'sunriseTime': 1579648260,
 'sunsetTime': 1579685760,
 'moonPhase': 0.92,
 'precipIntensity': 0.2701,
 'precipIntensityMax': 1.8801,
 'precipIntensityMaxTime': 1579669200,
 'precipProbability': 0.94,
 'precipType': 'rain',
 'temperatureHigh': 8.08,
 'temperatureHighTime': 1579679940,
 'temperatureLow': 5.4,
 'temperatureLowTime': 1579709340,
 'apparentTemperatureHigh': 6.33,
 'apparentTemperatureHighTime': 1579680480,
 'apparentTemperatureLow': 4.01,
 'apparentTemperatureLowTime': 1579710180,
 'dewPoint': 4.89,
 'humidity': 0.93,
 'pressure': 1022.5,
 'windSpeed': 1.76,
 'windGust': 4.81,
 'windGustTime': 1579676340,
 'windBearing': 344,
 'cloudCover': 0.99,
 'uvIndex': 3,
 'uvIndexTime': 1579667160,
 'visibility': 14.689,
 'ozone': 297.3,
 'temperatureMin': 4.04,
 'temperatureMinTime': 1579633440,
 'temperatureMax': 8.08,
 'temperatureMaxTime': 1579679940,
 'apparentTemperatureMin': 2.93,
 '

In [22]:
df_tMax.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.1824,116.4142,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# pull data from darksky weather API
do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'APIKEY'

# Number of seconds in one day
one_day = 86400


# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        #print(t, type(t))
        time_values = []
        
        # Create range of time stamps going back 14 days from date column
        for time_loop in range(15):
            the_stamp = t - (one_day * time_loop)
            time_values.append(str(the_stamp))
        #print(time_values)
        
        # Start iterating through the rows (locations)
        for y in range(df_tMax['1/22/20'].values.size):
            
            tMax_data = []
            tMin_data = []
            humidity_data = []
            uv_data = []
            cloud_data = []
            precip_data = []
            dew_data = []
            pressure_data = []
            wind_data = []
            ozone_data = []
            #summary_data = ''
            
                        
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
            #print (latitude, longitude)
            
            # Start iterating through the time stamps going back 14 days
            for z in time_values:
                
                # Building the URL for the API get
                url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + z
                url = url + '?exclude=currently,flags&units=si'
                
                # Getting the API call
                response = requests.get(url)
                
                # Putting the API response into the JSON thing
                info = json.loads(response.content)
                # Making a variable to more easily acccess JSON response data
                easy_info = info['daily']['data'][0]
                
                # Reading the JSON data
                tMax = easy_info['temperatureHigh']
                tMin = easy_info['temperatureLow']
                hum = easy_info['humidity'] * 100
                uvee = easy_info['uvIndex']
                clouds = easy_info['cloudCover'] * 100
                precip = easy_info['precipProbability'] * 100
                dew = easy_info['dewPoint']
                pressure = easy_info['pressure']
                wind = easy_info['windSpeed']
                ozone = easy_info['ozone']
                #summary_data = summary_data + ", " + easy_info['summary']
                
                # Recording the JSON data into the arrays 
                tMax_data.append(tMax)
                tMin_data.append(tMin)
                humidity_data.append(hum)
                uv_data.append(uvee)
                cloud_data.append(clouds)
                precip_data.append(precip)
                dew_data.append(dew)
                pressure_data.append(pressure)
                wind_data.append(wind)
                ozone_data.append(ozone)
            
            
            # Recording the medians of the preceding 14 days into the respective dataframes
            df_tMax.at[y, x] = statistics.median(tMax_data)
            df_tMin.at[y, x] = statistics.median(tMin_data)
            df_humidity.at[y, x] = statistics.median(humidity_data)
            df_uvIndex.at[y, x] = statistics.median(uv_data)
            df_cloud.at[y, x] = statistics.median(cloud_data)
            df_precipprob.at[y, x] = statistics.median(precip_data)
            df_dewpoint.at[y, x] = statistics.median(dew_data)
            df_pressure.at[y, x] = statistics.median(pressure_data)
            df_windspeed.at[y, x] = statistics.median(wind_data)
            df_ozone.at[y, x] = statistics.median(ozone_data)
            #df_summaries.at[y, x] = summary_data

                      
            # API has a daily limit, so we're just taking a break here
            if (y == 10):
                break
    if (x == '1/22/20'):
        break


In [24]:
# pull data from darksky weather API
# after the test run, I'm taking the breaks away, to max out the API for the day.
# I'm starting with removing the breaks in the y loop, which iterates through the rows.
# I'm setting the loops to pick up where the cell above left off after the test run.

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = 'APIKEY'

# Number of seconds in one day
one_day = 86400


# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        #print(t, type(t))
        time_values = []
        
        # Create range of time stamps going back 14 days from date column
        for time_loop in range(15):
            the_stamp = t - (one_day * time_loop)
            time_values.append(str(the_stamp))
        #print(time_values)
        
        # Start iterating through the rows (locations)
        # before, y went to 10. So here, we're starting with 11.
        for y in range(11, df_tMax['1/22/20'].values.size):
            
            tMax_data = []
            tMin_data = []
            humidity_data = []
            uv_data = []
            cloud_data = []
            precip_data = []
            dew_data = []
            pressure_data = []
            wind_data = []
            ozone_data = []
            #summary_data = ''
            
                        
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
            #print (latitude, longitude)
            
            # Start iterating through the time stamps going back 14 days
            for z in time_values:
                
                # Building the URL for the API get
                url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + z
                url = url + '?exclude=currently,flags&units=si'
                
                # Getting the API call
                response = requests.get(url)
                
                # Putting the API response into the JSON thing
                info = json.loads(response.content)
                # Making a variable to more easily acccess JSON response data
                easy_info = info['daily']['data'][0]
                
                # Reading the JSON data
                tMax = easy_info['temperatureHigh']
                tMin = easy_info['temperatureLow']
                hum = easy_info['humidity'] * 100
                uvee = easy_info['uvIndex']
                clouds = easy_info['cloudCover'] * 100
                precip = easy_info['precipProbability'] * 100
                dew = easy_info['dewPoint']
                pressure = easy_info['pressure']
                wind = easy_info['windSpeed']
                ozone = easy_info['ozone']
                #summary_data = summary_data + ", " + easy_info['summary']
                
                # Recording the JSON data into the arrays 
                tMax_data.append(tMax)
                tMin_data.append(tMin)
                humidity_data.append(hum)
                uv_data.append(uvee)
                cloud_data.append(clouds)
                precip_data.append(precip)
                dew_data.append(dew)
                pressure_data.append(pressure)
                wind_data.append(wind)
                ozone_data.append(ozone)
            
            
            # Recording the medians of the preceding 14 days into the respective dataframes
            df_tMax.at[y, x] = statistics.median(tMax_data)
            df_tMin.at[y, x] = statistics.median(tMin_data)
            df_humidity.at[y, x] = statistics.median(humidity_data)
            df_uvIndex.at[y, x] = statistics.median(uv_data)
            df_cloud.at[y, x] = statistics.median(cloud_data)
            df_precipprob.at[y, x] = statistics.median(precip_data)
            df_dewpoint.at[y, x] = statistics.median(dew_data)
            df_pressure.at[y, x] = statistics.median(pressure_data)
            df_windspeed.at[y, x] = statistics.median(wind_data)
            df_ozone.at[y, x] = statistics.median(ozone_data)
            #df_summaries.at[y, x] = summary_data

                      
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
    if (x == '1/22/20'):
        break


KeyError: 'daily'

In [28]:
# pull data from darksky weather API
# after the test run, I'm taking the breaks away, to max out the API for the day.
# Taking away all breaks and crossing fingers

do_not_include = ['Province/State', 'Country/Region', 'Lat', 'Long']

#Darksky API key
API_KEY = '35b9c12571882165902f56587063a264'

# Number of seconds in one day
one_day = 86400

# Creating backup counter to break, just in case
counter = 0

# Start iterating through the date columns
for x in df_tMax.columns.values:
    
    # Skip the columns that are not dates
    if (x not in do_not_include):
        
        # Create Unix time stamp out of the date column
        t = pd.to_datetime(df_tMax[x].name)
        t = int(t.value / 10**9)
        #print(t, type(t))
        time_values = []
        
        # Create range of time stamps going back 14 days from date column
        for time_loop in range(15):
            the_stamp = t - (one_day * time_loop)
            time_values.append(str(the_stamp))
        #print(time_values)
        
        # Start iterating through the rows (locations)
        # before, y went to 10. So here, we're starting with 11.
        for y in range(11, df_tMax['1/22/20'].values.size):
            
            tMax_data = []
            tMin_data = []
            humidity_data = []
            uv_data = []
            cloud_data = []
            precip_data = []
            dew_data = []
            pressure_data = []
            wind_data = []
            ozone_data = []
            #summary_data = ''
            
                        
            latitude = str(df_tMax.iloc[y][2])
            longitude = str(df_tMax.iloc[y][3])
            #print (latitude, longitude)
            
            # Start iterating through the time stamps going back 14 days
            for z in time_values:
                
                # Building the URL for the API get
                url = 'https://api.darksky.net/forecast/' + API_KEY + '/' + latitude + "," + longitude + ',' + z
                url = url + '?exclude=currently,flags&units=si'
                
                # Getting the API call
                response = requests.get(url)
                
                # counting API calls
                counter = counter + 1
                
                # Putting the API response into the JSON thing
                info = json.loads(response.content)
                # Making a variable to more easily acccess JSON response data
                easy_info = info['daily']['data'][0]
                
                # Reading the JSON data
                tMax = easy_info['temperatureHigh']
                tMin = easy_info['temperatureLow']
                hum = easy_info['humidity'] * 100
                uvee = easy_info['uvIndex']
                clouds = easy_info['cloudCover'] * 100
                precip = easy_info['precipProbability'] * 100
                dew = easy_info['dewPoint']
                pressure = easy_info['pressure']
                wind = easy_info['windSpeed']
                ozone = easy_info['ozone']
                #summary_data = summary_data + ", " + easy_info['summary']
                
                # Recording the JSON data into the arrays 
                tMax_data.append(tMax)
                tMin_data.append(tMin)
                humidity_data.append(hum)
                uv_data.append(uvee)
                cloud_data.append(clouds)
                precip_data.append(precip)
                dew_data.append(dew)
                pressure_data.append(pressure)
                wind_data.append(wind)
                ozone_data.append(ozone)
            
            
            # Recording the medians of the preceding 14 days into the respective dataframes
            df_tMax.at[y, x] = statistics.median(tMax_data)
            df_tMin.at[y, x] = statistics.median(tMin_data)
            df_humidity.at[y, x] = statistics.median(humidity_data)
            df_uvIndex.at[y, x] = statistics.median(uv_data)
            df_cloud.at[y, x] = statistics.median(cloud_data)
            df_precipprob.at[y, x] = statistics.median(precip_data)
            df_dewpoint.at[y, x] = statistics.median(dew_data)
            df_pressure.at[y, x] = statistics.median(pressure_data)
            df_windspeed.at[y, x] = statistics.median(wind_data)
            df_ozone.at[y, x] = statistics.median(ozone_data)
            #df_summaries.at[y, x] = summary_data

                      
            # API has a daily limit, so we're just taking a break here
            #if (y == 10):
             #   break
    
    # By my calculation, there should be 179775 API calls. I've created a buffer of 5 with the counter
    if (counter == 179780):
        break
     


In [12]:
df_tMax.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_tMin.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_humidity.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,80,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,79,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,60,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,82,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,88,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,70,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_uvIndex.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_cloud.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,83,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,67,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,96,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_precipprob.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_dewpoint.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,-9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,-6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_pressure.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,1026,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1021,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1023,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,1020,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,1019,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1015,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1027,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_windspeed.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_ozone.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20
0,Anhui,Mainland China,31.82570,117.2264,278,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beijing,Mainland China,40.18240,116.4142,356,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chongqing,Mainland China,30.05720,107.8740,271,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fujian,Mainland China,26.07890,117.9874,253,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,322,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Guangdong,Mainland China,23.34170,113.4244,246,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Guangxi,Mainland China,23.82980,108.7881,246,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Guizhou,Mainland China,26.81540,106.8748,251,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hainan,Mainland China,19.19590,109.7453,237,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hebei,Mainland China,38.04280,114.5149,341,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# writing CSVs of what I've got so far

df_tMax.to_csv('./csv/tMax.csv', index=False)

df_tMin.to_csv('./csv/tMin.csv', index=False)

df_humidity.to_csv('./csv/humidity.csv', index=False)

df_uvIndex.to_csv('./csv/uv.csv', index=False)

df_cloud.to_csv('./csv/cloud.csv', index=False)

df_precipprob.to_csv('./csv/precip.csv', index=False)

df_dewpoint.to_csv('./csv/dew.csv', index=False)

df_pressure.to_csv('./csv/pressure.csv', index=False)

df_windspeed.to_csv('./csv/wind.csv', index=False)

df_ozone.to_csv('./csv/ozone.csv', index=False)
